### Quick Description

This notebook calculates the toxicity score from each sentence in a subtitles' file. Finally, it persists the data in a DataFrame like output.

In [3]:
import yaml
import glob
import os
import pandas as pd
import time
import re
import logging
import sys

from tqdm import tqdm
from flair.models import TextClassifier
from flair.data import Sentence


sys.path.append("../utils")
from toxicity_api_communication import get_toxicity_score

/home/luiznery/anaconda3/envs/fake/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
filepaths = yaml.load(open("../config/filepaths.yaml"))
credentials = yaml.load(open("../config/credentials.yaml"))["perspective-api"]

/tmp/ipykernel_359549/1089545284.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  filepaths = yaml.load(open("../config/filepaths.yaml"))
/tmp/ipykernel_359549/1089545284.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credentials = yaml.load(open("../config/credentials.yaml"))["perspective-api"]


In [5]:
def calculate_toxicity_scores(credentials, save_dir):
    classifier = TextClassifier.load('en-sentiment')
    filenames = [file.split("/")[-1] for file in glob.glob(os.path.join(save_dir, "*"))]

    for filename in tqdm(filenames, total=len(filenames)):
        df_sentences = pd.read_csv(os.path.join(save_dir, filename))
        for i, row in df_sentences.iterrows():
            sentence = Sentence(row.text)
            classifier.predict(sentence)

            df_sentences.loc[i, "flair_score"] = (
                1 - sentence.labels[0]._score
                if sentence.labels[0]._value == "POSITIVE"
                else 1 + sentence.labels[0]._score
            )
            
        # Because 2 is the absolute maximum value
        df_sentences["flair_score"] = (
            (df_sentences["flair_score"] - df_sentences["flair_score"].min())
            / (2 - df_sentences["flair_score"].min())
        )

        df_sentences.to_csv(os.path.join(save_dir, filename), index=False)

In [6]:
calculate_toxicity_scores(
    credentials,
    filepaths["coraal_scored"]
)

2022-06-03 17:03:24,113 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpnvmqxj1g


100%|██████████| 265512723/265512723 [00:24<00:00, 10713305.78B/s]

2022-06-03 17:03:49,469 copying /tmp/tmpnvmqxj1g to cache at /home/luiznery/.flair/models/sentiment-en-mix-distillbert_4.pt
2022-06-03 17:03:49,622 removing temp file /tmp/tmpnvmqxj1g
2022-06-03 17:03:49,651 loading file /home/luiznery/.flair/models/sentiment-en-mix-distillbert_4.pt



Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 15.2kB/s]
Downloading: 100%|██████████| 483/483 [00:00<00:00, 272kB/s]
Downloading: 100%|██████████| 226k/226k [00:00<00:00, 455kB/s]  
Downloading: 100%|██████████| 455k/455k [00:00<00:00, 734kB/s]  
100%|██████████| 108/108 [25:01<00:00, 13.90s/it]
